# Video Moderation with Human-in-the-loop

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

Amazon A2I allows you to create your workflows for ML models built on Amazon SageMaker or any other customized workflow. Using Amazon A2I, you can allow human reviewers to step in when a model cannot make a high-confidence prediction or audit its predictions on an ongoing basis. Learn more [here](https://aws.amazon.com/augmented-ai/).

In this tutorial, we will show how you can use Amazon A2I to customize a UI template and use it to review Rekognition's Video moderation result. 

For more in depth instructions, visit this [website](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-getting-started.html).

![image-moderation-hitl-arc](../images/video-moderation-hitl-architecture.png)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Moderate video using Rekognition video moderation API](#step2)
- [Step 3: Set up A2I for human review](#step3)
    - [Step 3.1: Create Work Team](#step31)
    - [Step 3.2: Create Human Task UI](#step32)
    - [Step 3.3: Creating the Flow Definition](#step33)
- [Step 4: Moderate image with human-in-the-loop](#step4)
- [Step 5: Check A2I generated result in S3](#step5)

# Step 1: Setup Notebook <a id="step1"></a>

In [ ]:
# First, let's get the latest installations of our dependencies
%pip install --upgrade pip
%pip install boto3 --upgrade
%pip install IPython --upgrade

In [ ]:
import boto3
import botocore
import sagemaker as sm
import os
import datetime
from IPython.display import Video
from IPython.display import HTML, display
import uuid
import json
import time

# variables
data_bucket = sm.Session().default_bucket()
region = boto3.session.Session().region_name

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sm.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3', region_name=region)
rekognition=boto3.client('rekognition', region_name=region)
sagemaker = boto3.client('sagemaker', region_name=region)
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', region_name=region)

For this lab, we will send a moderation demo video located at datasets/moderation-video.mp4 to Rekognition. Run the below cell to preview the video.

In [ ]:
Video('../datasets/moderation-video.mp4')

Now, let's upload the video to the default S3 bucket for Rekognition to access.

In [ ]:
s3_key = 'content-moderation-im/video-moderation/moderation-video.mp4'
s3.upload_file('../datasets/moderation-video.mp4', data_bucket, s3_key)

# Step 2: Moderate video using Rekognition moderation API  <a id="step2"></a>
Call Rekognition Content Moderation API to detect inappropriate information in the video. Rekognition Video moderation API is an asynchronize API that will start a job managed by Rekognition. We will receive a job_id back when we start the job. Then use iteration logic to heart-beat the get_content_moderation API to check job status until the job completes.

In [ ]:
startModerationLabelDetection = rekognition.start_content_moderation(
        Video={
            'S3Object': {
                'Bucket': data_bucket,
                'Name': s3_key,
            }
        },
        #MinConfidence=min_confidence,
)

moderationJobId = startModerationLabelDetection['JobId']
display("Job Id: {0}".format(moderationJobId))

getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP'
)

while(getContentModeration['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')

    getContentModeration = rekognition.get_content_moderation(
    JobId=moderationJobId,
    SortBy='TIMESTAMP')

display(getContentModeration['JobStatus'])

In [ ]:
getContentModeration

As we can see in the Rekognition response, the get_content_moderation API returns a list of labels under the "ModerationLabels" field, a list of items each contains:
- Timestamp (millisecond): when inapproperiate information found within the video
- ModerationLabel:
    - Confidence: Confidence score of the label
    - Name: second level moderation category name
    - ParentName: top-level moderation category name

There are multiple items returned from Rekognition, and hard to review this result by looking at the data directly. In the next step, we will set up A2I so the human reviewers can check these results quickly.

# Step 3: Set up A2I for human review <a id="step3"></a>

#### Setup Bucket CORS

**Important**: The bucket you specify for `data_bucket` must have CORS enabled. You can enable CORS by adding a policy similar to the following to your Amazon S3 bucket. To learn how to add CORS to an S3 bucket, see [CORS Permission Requirement](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html#a2i-cors-update) in the Amazon A2I documentation. 


```
[{
   "AllowedHeaders": [],
   "AllowedMethods": ["GET"],
   "AllowedOrigins": ["*"],
   "ExposeHeaders": []
}]
```

If you do not add a CORS configuration to the S3 buckets that contains your image input data, human review tasks for those input data objects will fail. 

Print out the default S3 bucket name using the below cell if you don't know which bucket is in use.

In [ ]:
print('The default SageMaker Studio S3 bucket name: ' + data_bucket)

Run the below cell to enable CORS to the default S3 bucket

In [ ]:
cors_configuration = {
    'CORSRules': [{
       "AllowedHeaders": [],
       "AllowedMethods": ["GET"],
       "AllowedOrigins": ["*"],
       "ExposeHeaders": []
    }]
}

s3.put_bucket_cors(Bucket=data_bucket,
                   CORSConfiguration=cors_configuration)

You can execute all the cells in Step 3 to provision A2I components programmatically. Or you can follow the [instruction in the Content Moderation Workshop](https://catalog.workshops.aws/content-moderation/02-video-moderation/02-video-moderation-with-a2i) to set up them using the AWS console if you have already done the setups through the console. You can copy/paste the workflow ARN to the below cell and run it, then proceed to Step 4.

In [ ]:
# Copy/paste the workflow ARN to set the below variable if you have already done A2I setup using the AWS console
workflow_definition_arn = ''

## Step 3.1: Create Work Team <a id="step31"></a>
 A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers. When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job. 

This lab assumes that you already have the workforce team in the same region. If you don't have a workforce team in the current region, follow [Step 1](https://catalog.us-east-1.prod.workshops.aws/workshops/1ece9ffd-4c24-4e66-b42a-0c0e13b0f668/en-US/content-moderation/02-video-moderation/02-video-moderation-with-a2i#step-1:-create-a-private-team-in-aws-console-(you-can-skip-this-step-if-you-already-have-a-private-work-team-in-the-region)) and [Step 2](https://catalog.us-east-1.prod.workshops.aws/workshops/1ece9ffd-4c24-4e66-b42a-0c0e13b0f668/en-US/content-moderation/02-video-moderation/02-video-moderation-with-a2i#step-2:-activate-a2i-user-account) on the Workshop website to set it up in the AWS console.

[More documents](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html ) about how to set up a workforce team for A2I.

In [ ]:
# get the existing workforce arn
work_team_arn = sagemaker.list_workteams()["Workteams"][0]["WorkteamArn"]
work_team_arn

## Step 3.2: Create Human Task UI <a id="step32"></a>
Create a human task UI resource, giving a UI template in liquid HTML. This template will be rendered to the human workers whenever a human loop is required.
We will use a customized UI template that is compatible with the dictionary data will prepare in the following steps.
The customized UI template locates in the same folder: a2i-video-moderation-custom-template.html. It is a static web page with HTML, CSS and JavaScripts. You can open the file and check the detailed logic there.

In [ ]:
# read the UI template from the HTML file
template = ""
with open('a2i-video-moderation-custom-template.html','r') as f:
    template = f.read()

human_task_name = "a2i-rekognition-video-moderation-custom-ui"

# Create A2I UI template
resp = sagemaker.create_human_task_ui(
        HumanTaskUiName=human_task_name,
        UiTemplate={'Content': template})

In [ ]:
# Keep the new UI template ARN in a variable
ui_template_arn = resp["HumanTaskUiArn"]
ui_template_arn

## Step 3.3: Creating the Flow Definition <a id="step33"></a>
In this section, we're going to create a flow definition. Flow Definitions allow us to specify:
- The workforce that your tasks will be sent to. 
- The instructions that your workforce will receive. This is called a worker task template. 
- The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks. Where your output data will be stored.

This demo is going to use the API, but you can optionally create this workflow definition in the console as well. For more details and instructions, see [this document](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html).

In [ ]:
flow_def_name = "content-moderation-im-video-moderation-custom-ui-workflow"

resp = sagemaker.create_flow_definition(
        FlowDefinitionName= flow_def_name,
        RoleArn= role,
        HumanLoopConfig= {
            "WorkteamArn": work_team_arn,
            "HumanTaskUiArn": ui_template_arn,
            "TaskCount": 1,
            "TaskDescription": "A2I Rekognition video moderation workflow",
            "TaskTitle": "Content Moderation Video Human Loop Task"
        },
        OutputConfig={
            "S3OutputPath" : f's3://{data_bucket}/a2i/output/'
        }
    )

workflow_definition_arn = resp['FlowDefinitionArn']
workflow_definition_arn

# Step 4: Moderate video with human review <a id="step4"></a>

The A2I custom UI template allows us to decide when to trigger A2I. It allows us to customize the business rule to support more complex business requirements. In the below code, we will start A2I only when the moderation top category is `Alcoholic Beverages` or `Barechested Male` or `Drinking` with a confidence score greater than 50%.

In [ ]:
# A function to prepare the data in the format the custom UI expects
# This function will only include the labels in "Alcoholic Beverages","Barechested Male" or "Drinking" and confidence > 80%
def generate_a2i_payload(rek_response):
    failed_conditions = []
    for item in rek_response["ModerationLabels"]:
        if item["ModerationLabel"]["Name"] in ["Alcoholic Beverages","Barechested Male", "Drinking"] and item["ModerationLabel"]["Confidence"] > 80:
            failed_conditions.append(item)
    
    return {
                "InputContent": json.dumps({
                    "Results": {
                        "ConditionMissed": failed_conditions,
                    },
                    "s3":{
                        "url": f's3://{data_bucket}/{s3_key}'
                    },
                    "VideoName": "Tear of Steel Trailer",
                })
            }

In [ ]:
# the data will send to A2I
a2i_payload = generate_a2i_payload(getContentModeration)

In [ ]:
display(a2i_payload)

As you can see, the function filtered the moderation labels and only kept 2 of them in the list. We will send this list of moderated labels to A2I for human review. 

We now start an A2I human review task by sending this dataset and the other information to the API.

In [ ]:
human_loop_name = f'cm-im-video-{str(uuid.uuid4())}'

start_loop_response = a2i_runtime_client.start_human_loop(
            HumanLoopName=human_loop_name,
            FlowDefinitionArn=workflow_definition_arn,
            HumanLoopInput=a2i_payload)
human_loop_arn = start_loop_response["HumanLoopArn"]


Check Human Loop status

In [ ]:
a2i_response = a2i_runtime_client.describe_human_loop(HumanLoopName=human_loop_name)
print(f'\nHuman Loop Name: {human_loop_name}')
print(f'Human Loop Status: {a2i_response["HumanLoopStatus"]}')

A human loop task is now sent to A2I and ready for us to review. You can find the A2I console URL from SageMaker console under Augment AI -> Human review workforces under the "private" tab. 

Or use the below code to print out the A2I console URL:

In [ ]:
workteamName = work_team_arn[work_team_arn.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

Login to the A2I console using the username/password you received earlier when setting up the Workforce team. You should see the task "Content Moderation Video Human Loop Task" in the list. Choose the item and click on the "start working" button on the top right to start viewing the moderation result. You will see a review page like the below screenshot:

![a2i video moderation](../images/video-moderation-hitl-a2i.png)

This UI is rendered using the customized template we imported in the previous steps. You can modify the HTML to change the layout or UI logic of the page. 

As a reviewer, you can review the moderation categories, then click on the "Submit" button. A2I will store the reviewer input in the S3 output folder.

Now, the human loop status should updated to "completed".

In [ ]:
a2i_resp = a2i_runtime_client.describe_human_loop(HumanLoopName=human_loop_name)
a2i_resp

# Step 5 Check A2I generated JSON <a id="step5"></a>
Now, let's download the A2I output file and print it out:

In [ ]:
s3.download_file(data_bucket, a2i_resp["HumanLoopOutput"]["OutputS3Uri"].replace(f's3://{data_bucket}/',''), 'a2i-output-video-moderation.json')

The `humanAnswers` field in the JSON file contains the reviewer's input. The `inputContent` field contains the original data sent to A2I.

In [ ]:
with open('a2i-output-video-moderation.json','r') as f:
    print(json.dumps(json.loads(f.read()), indent=2))

# Cleanup

In [ ]:
# Delete A2I flow definition: this step may take a few minutes if human loop tasks in the queue and not processed
sagemaker.delete_flow_definition(FlowDefinitionName=flow_def_name)

In [ ]:
# Delete A2I custom human task UI
sagemaker.delete_human_task_ui(HumanTaskUiName=human_task_name)

In [ ]:
# Delete sample video from S3 bucket
s3.delete_object(Bucket=data_bucket, Key=s3_key)

# Conclusion
In this lab, we moderated a video using Rekognition Video Moderation API. And set up a customized A2I UI with customized rules to review the moderation result.